In [ ]:
!git clone https://github.com/Gibbsdavidl/gssnng
!pip install -e gssnng

In [ ]:
from gssnng import score_cells
import scanpy as sc
import matplotlib
%matplotlib inline

In [ ]:
q = sc.read_h5ad('gssnng/gssnng/test/data/pbmc3k_processed.h5ad')

In [ ]:
sc.pl.umap(q, color='louvain')

In [ ]:
# recompute the nearest neighbor graph to give plenty of neighbors for each cell
sc.pp.neighbors(q, n_neighbors=32)

In [ ]:
score_cells.with_gene_sets(
        adata=q,
        gene_set_file="gssnng/gssnng/test/data/cibersort_lm22.gmt",
        score_method="robust_std",
        method_params=dict(),
        samp_neighbors=29,
        noise_trials=0,
        keys_added=None
    )

In [ ]:
q.obs

In [ ]:
sc.pl.umap(q, color=['louvain','T.cells.CD8'], wspace=0.35)